<a href="https://colab.research.google.com/github/jauger4/ufc_judge/blob/main/ufc_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
#load the dataset
ufc = pd.read_csv('large_dataset.csv')
ufc.head()

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,weight_diff,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff
0,UFC Fight Night: Ribas vs. Namajunas,Amanda Ribas,Rose Namajunas,Blue,Women's Flyweight,0,Women,Decision - Unanimous,5,5.0,...,0.00,2.54,0.94,-0.11,-0.01,0.04,-0.02,0.26,0.2,0.69
1,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,-13.16,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75
2,UFC Fight Night: Ribas vs. Namajunas,Edmen Shahbazyan,AJ Dobson,Red,Middleweight,0,Men,KO/TKO,1,3.0,...,0.00,-2.54,-0.69,-1.22,0.06,-0.37,-0.01,-0.02,0.3,0.57
3,UFC Fight Night: Ribas vs. Namajunas,Payton Talbott,Cameron Saaiman,Red,Bantamweight,0,Men,KO/TKO,2,3.0,...,0.00,7.62,2.73,-0.60,0.08,-0.28,0.00,0.43,-0.2,-0.91
4,UFC Fight Night: Ribas vs. Namajunas,Billy Quarantillo,Youssef Zalal,Blue,Featherweight,0,Men,Submission,2,3.0,...,0.00,-5.08,4.48,3.84,0.07,-0.11,-0.22,0.01,-0.2,-1.04


In [ ]:
#filter instances that do not include decisions
ufc_decisions = ufc[ufc['method'].str.contains('Decision', case=False, na=False)].copy()

ufc_decisions.head()

,event_name,r_fighter,b_fighter,winner,weight_class,is_title_bout,gender,method,finish_round,total_rounds,...,weight_diff,reach_diff,SLpM_total_diff,SApM_total_diff,sig_str_acc_total_diff,td_acc_total_diff,str_def_total_diff,td_def_total_diff,sub_avg_diff,td_avg_diff
0,UFC Fight Night: Ribas vs. Namajunas,Amanda Ribas,Rose Namajunas,Blue,Women's Flyweight,0,Women,Decision - Unanimous,5,5.0,...,0.00,2.54,0.94,-0.11,-0.01,0.04,-0.02,0.26,0.2,0.69
1,UFC Fight Night: Ribas vs. Namajunas,Karl Williams,Justin Tafa,Red,Heavyweight,0,Men,Decision - Unanimous,3,3.0,...,-13.16,12.70,-1.22,-3.32,-0.02,0.50,0.13,0.50,0.2,4.75
6,UFC Fight Night: Ribas vs. Namajunas,Kurt Holobaugh,Trey Ogden,Blue,Lightweight,0,Men,Decision - Unanimous,3,3.0,...,0.00,-5.08,0.99,0.48,-0.04,0.13,-0.10,-0.54,0.3,-0.93
8,UFC Fight Night: Ribas vs. Namajunas,Miles Johns,Cody Gibson,Red,Bantamweight,0,Men,Decision - Unanimous,3,3.0,...,0.00,-12.70,-0.72,-1.53,0.04,-0.06,0.11,0.17,-0.2,-0.13
9,UFC Fight Night: Ribas vs. Namajunas,Jarno Errens,Steven Nguyen,Red,Featherweight,0,Men,Decision - Unanimous,3,3.0,...,0.00,0.00,-3.71,-4.00,0.06,-1.00,0.03,-0.56,0.3,-0.28


In [ ]:
#drop unnecessary or null attributes
cols = ufc_decisions.columns.to_list()
diff_cols = [col for col in cols if '_diff' in col]

cols_to_drop = [
    'event_name',
    'referee',
    'r_fighter',
    'b_fighter',
    'total_rounds',
    'finish_round',
    'time_sec',
    'r_stance',
    'b_stance',
] + diff_cols

ufc_decisions = ufc_decisions.drop(columns = cols_to_drop)
ufc_decisions.dropna(inplace = True)
ufc_decisions.head()

,winner,weight_class,is_title_bout,gender,method,r_kd,r_sig_str,r_sig_str_att,r_sig_str_acc,r_str,...,b_weight,b_reach,b_SLpM_total,b_SApM_total,b_sig_str_acc_total,b_td_acc_total,b_str_def_total,b_td_def_total,b_sub_avg,b_td_avg
0,Blue,Women's Flyweight,0,Women,Decision - Unanimous,0,83,270,0.30,137,...,56.70,165.10,3.69,3.51,0.41,0.47,0.63,0.59,0.5,1.38
1,Red,Heavyweight,0,Men,Decision - Unanimous,0,40,61,0.65,123,...,119.75,187.96,4.09,5.02,0.54,0.00,0.47,0.50,0.0,0.00
6,Blue,Lightweight,0,Men,Decision - Unanimous,0,29,68,0.42,129,...,70.31,182.88,3.38,3.59,0.47,0.23,0.58,1.00,0.6,1.64
8,Red,Bantamweight,0,Men,Decision - Unanimous,0,25,79,0.31,64,...,61.23,180.34,4.00,4.08,0.43,0.28,0.57,0.68,0.2,1.28
9,Red,Featherweight,0,Men,Decision - Unanimous,1,105,207,0.50,106,...,65.77,185.42,7.15,7.36,0.40,1.00,0.60,0.84,0.0,0.28


In [ ]:
#split data by corner color to flip and concatenate to original df
cols_new = ufc_decisions.columns.to_list()

r_cols = [col for col in cols_new if col.startswith('r_')]
b_cols = [col for col in cols_new if col.startswith('b_')]

# Identify columns that exist for both 'r_' and 'b_' prefixes
# For example, if 'r_age' exists, check if 'b_age' exists
flippable_r_cols = []
for r_col_name in r_cols:
    corresponding_b_col_name = r_col_name.replace('r_', 'b_')
    if corresponding_b_col_name in b_cols:
        flippable_r_cols.append(r_col_name)

ufc_decisions_flipped = ufc_decisions.copy()

# Swap the 'r_' and 'b_' columns in the flipped DataFrame
for r_col_name in flippable_r_cols:
    b_col_name = r_col_name.replace('r_', 'b_')
    temp = ufc_decisions_flipped[r_col_name].copy()
    ufc_decisions_flipped[r_col_name] = ufc_decisions_flipped[b_col_name]
    ufc_decisions_flipped[b_col_name] = temp

# Flip the values of the winner column
ufc_decisions_flipped['winner'] = ufc_decisions_flipped['winner'].replace({'Red': 'Blue', 'Blue': 'Red'})

ufc_decisions = pd.concat([ufc_decisions, ufc_decisions_flipped], ignore_index=True)

ufc_decisions.head()

,winner,weight_class,is_title_bout,gender,method,r_kd,r_sig_str,r_sig_str_att,r_sig_str_acc,r_str,...,b_weight,b_reach,b_SLpM_total,b_SApM_total,b_sig_str_acc_total,b_td_acc_total,b_str_def_total,b_td_def_total,b_sub_avg,b_td_avg
0,Blue,Women's Flyweight,0,Women,Decision - Unanimous,0,83,270,0.30,137,...,56.70,165.10,3.69,3.51,0.41,0.47,0.63,0.59,0.5,1.38
1,Red,Heavyweight,0,Men,Decision - Unanimous,0,40,61,0.65,123,...,119.75,187.96,4.09,5.02,0.54,0.00,0.47,0.50,0.0,0.00
2,Blue,Lightweight,0,Men,Decision - Unanimous,0,29,68,0.42,129,...,70.31,182.88,3.38,3.59,0.47,0.23,0.58,1.00,0.6,1.64
3,Red,Bantamweight,0,Men,Decision - Unanimous,0,25,79,0.31,64,...,61.23,180.34,4.00,4.08,0.43,0.28,0.57,0.68,0.2,1.28
4,Red,Featherweight,0,Men,Decision - Unanimous,1,105,207,0.50,106,...,65.77,185.42,7.15,7.36,0.40,1.00,0.60,0.84,0.0,0.28


In [ ]:
ufc_decisions_flipped.head()

,winner,weight_class,is_title_bout,gender,method,r_kd,r_sig_str,r_sig_str_att,r_sig_str_acc,r_str,...,b_weight,b_reach,b_SLpM_total,b_SApM_total,b_sig_str_acc_total,b_td_acc_total,b_str_def_total,b_td_def_total,b_sub_avg,b_td_avg
0,Red,Women's Flyweight,0,Women,Decision - Unanimous,0,93,270,0.30,169,...,56.70,167.64,4.63,3.40,0.41,0.51,0.63,0.85,0.7,2.07
1,Blue,Heavyweight,0,Men,Decision - Unanimous,0,21,61,0.65,27,...,106.59,200.66,2.87,1.70,0.54,0.50,0.47,1.00,0.2,4.75
6,Red,Lightweight,0,Men,Decision - Unanimous,0,35,68,0.42,225,...,70.31,177.80,4.37,4.07,0.47,0.36,0.58,0.46,0.9,0.71
8,Blue,Bantamweight,0,Men,Decision - Unanimous,0,18,79,0.31,52,...,61.23,167.64,3.28,2.55,0.43,0.22,0.57,0.85,0.0,1.15
9,Blue,Featherweight,0,Men,Decision - Unanimous,0,63,207,0.50,63,...,65.77,185.42,3.44,3.36,0.40,0.00,0.60,0.28,0.3,0.00


In [ ]:
print(ufc_decisions[['r_height', 'b_height', 'r_age', 'b_age']].dtypes)

r_height    float64
b_height    float64
r_age       float64
b_age       float64
dtype: object
